## Experimental Wallet functionality 

This is a little experiment to help track account balances for different currencies. This will likely find its way to the official API sooner or later

In [ ]:
#| default_exp wallet

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import asyncio
from typing import List, Optional
from dataclasses import dataclass
from sugar import AsyncChain
from sugar.token import Token
from sugar import get_async_chain, get_chain
from sugar.pool import LiquidityPool, Amount
from sugar.helpers import normalize_address
from sugar.price import  Price

In [ ]:
#| export

@dataclass
class TokenBalance:
    """Represents a token balance with metadata"""
    token: Token
    balance: float
    price_stable: float
    
    @property
    def balance_stable(self) -> float:
        """Computed property for stable currency value"""
        return self.balance * self.price_stable

# ERC-20 token ABI for balanceOf function
ERC20_ABI = [
    {
        "constant": True,
        "inputs": [{"name": "owner", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "", "type": "uint256"}],
        "type": "function"
    }
]


async def process_token_batch(self, token_batch, address, price_lookup=None):
    """Process a batch of tokens using concurrent async calls"""
    async def get_single_token_balance(token):
        """Get balance for a single token"""
        try:
            contract = self.web3.eth.contract(
                address=self.web3.to_checksum_address(token.token_address),
                abi=ERC20_ABI
            )
            
            balance_wei = await contract.functions.balanceOf(address).call()
            balance = balance_wei / (10 ** token.decimals)
            
            # Get stable price if available
            price_stable = price_lookup.get(token.token_address, 0.0) if price_lookup else 0.0
            
            return TokenBalance(
                token=token,
                balance=balance,
                price_stable=price_stable
            )
        except Exception:
            return None
    
    # Use asyncio.gather to make concurrent calls
    tasks = [get_single_token_balance(token) for token in token_batch]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    # Filter out None results and exceptions
    valid_results = []
    for result in results:
        if result is not None and not isinstance(result, Exception):
            valid_results.append(result)
    
    return valid_results

# Monkey patch AsyncChain to add get_token_balances method
async def get_token_balances(self: AsyncChain, tokens: Optional[List[Token]] = None, prices: Optional[List[Price]] = None,  address=None):
    """Get all token balances for a given address using batched requests
    
    This method uses asyncio.gather to make concurrent RPC calls in batches,
    which is significantly faster than sequential calls.
    
    Performance: ~16x faster than sequential method
    
    Args:
        address: The address to check balances for. If None, uses self.account.address
    
    Returns:
        List of TokenBalance objects with token info and stable currency values
    """
    if address is None:
        address = self.account.address
    
    balances, tokens = [],  tokens or await self.get_all_tokens()
    prices = prices or await self.get_prices(tokens)
    seen_addresses, erc20_tokens = set(), []
    
    # Build price lookup dictionary
    price_lookup = {}
    for price in prices:
        price_lookup[price.token.token_address] = price.price
    

    # Get ETH balance
    eth_balance = self.web3.from_wei(await self.web3.eth.get_balance(address), 'ether')
    eth_price = price_lookup.get('ETH', 0.0)
    
    # Create a Token object for ETH
    eth_token = Token(
        chain_id=self.chain_id,
        chain_name=self.name,
        token_address='ETH',
        symbol='ETH',
        decimals=18,
        listed=True,
        wrapped_token_address=None
    )
    
    balances.append(TokenBalance(
        token=eth_token,
        balance=float(eth_balance),
        price_stable=eth_price
    ))

    for token in tokens:
        if token.token_address != 'ETH' and token.listed and token.token_address not in seen_addresses:
            erc20_tokens.append(token)
            seen_addresses.add(token.token_address)

    # Process tokens in batches
    batch_size = 50  # Adjust based on RPC limits
    
    for i in range(0, len(erc20_tokens), batch_size):
        batch = erc20_tokens[i:i + batch_size]
        # Process batch using batch_requests
        batch_results = await process_token_batch(self, batch, address, price_lookup)
        
        # Add non-zero balances to results
        for result in batch_results:
            if result and result.balance > 0:
                balances.append(result)
    
    return balances

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()